# HSI Biopsy Data Analysis Example

This notebook demonstrates basic loading and analysis of hyperspectral imaging data from biopsy samples.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

# Add project root to path
sys.path.append('..')

# Import project modules
from src import preprocessing

## Data Loading

In this section, we would load the HSI data. For now, we'll create a synthetic example.

In [ ]:
# Create synthetic HSI data for demonstration
# In real applications, this would be loaded from actual data files

# Create a 100x100 spatial image with 50 spectral bands
rows, cols, bands = 100, 100, 50
wavelengths = np.linspace(400, 900, bands)  # wavelengths in nm

# Create synthetic HSI cube
hsi_data = np.zeros((rows, cols, bands))

# Add some synthetic spectral signatures
for i in range(rows):
    for j in range(cols):
        # Base spectrum - Gaussian centered at different wavelengths
        center = 400 + (i * 5) % 500  # different centers based on position
        hsi_data[i, j, :] = np.exp(-0.5 * ((wavelengths - center) / 50) ** 2)
        
        # Add noise
        hsi_data[i, j, :] += np.random.normal(0, 0.05, bands)

# Add a synthetic anomaly/region of interest
center_i, center_j = rows // 2, cols // 2
radius = 15

for i in range(center_i - radius, center_i + radius):
    for j in range(center_j - radius, center_j + radius):
        if 0 <= i < rows and 0 <= j < cols:
            if ((i - center_i) ** 2 + (j - center_j) ** 2) < radius ** 2:
                # Different spectral signature for the region of interest
                hsi_data[i, j, :] = np.exp(-0.5 * ((wavelengths - 700) / 30) ** 2) + np.random.normal(0, 0.05, bands)

## Visualization

Let's visualize the data by creating an RGB representation and looking at some spectral signatures.

In [ ]:
def create_rgb(hsi_data, wavelengths, r_band=650, g_band=550, b_band=450):
    """Create RGB image from hyperspectral data"""
    # Find closest wavelength bands
    r_idx = np.argmin(np.abs(wavelengths - r_band))
    g_idx = np.argmin(np.abs(wavelengths - g_band))
    b_idx = np.argmin(np.abs(wavelengths - b_band))
    
    # Create RGB image
    rgb = np.zeros((hsi_data.shape[0], hsi_data.shape[1], 3))
    rgb[:, :, 0] = hsi_data[:, :, r_idx]
    rgb[:, :, 1] = hsi_data[:, :, g_idx]
    rgb[:, :, 2] = hsi_data[:, :, b_idx]
    
    # Normalize to 0-1 range for display
    for i in range(3):
        channel_min = rgb[:, :, i].min()
        channel_max = rgb[:, :, i].max()
        rgb[:, :, i] = (rgb[:, :, i] - channel_min) / (channel_max - channel_min)
    
    return rgb

# Create and display RGB image
rgb_image = create_rgb(hsi_data, wavelengths)

plt.figure(figsize=(10, 8))
plt.imshow(rgb_image)
plt.title('RGB Representation of HSI Data')
plt.axis('off')
plt.colorbar(label='Intensity')
plt.show()

# Plot some example spectra
plt.figure(figsize=(12, 6))

# Normal tissue spectrum (from outside the ROI)
normal_i, normal_j = 10, 10
plt.plot(wavelengths, hsi_data[normal_i, normal_j, :], 'b-', label='Normal tissue')

# ROI spectrum (from inside the circle)
roi_i, roi_j = center_i, center_j
plt.plot(wavelengths, hsi_data[roi_i, roi_j, :], 'r-', label='Region of interest')

plt.xlabel('Wavelength (nm)')
plt.ylabel('Intensity')
plt.title('Spectral Signatures')
plt.legend()
plt.grid(True)
plt.show()

## Next Steps

In a real analysis workflow, next steps might include:

1. Spectral preprocessing (smoothing, normalization)
2. Dimensionality reduction (PCA, t-SNE)
3. Feature extraction
4. Classification or clustering
5. Validation against ground truth

These steps would be implemented as the project develops.